In [14]:
import time
start_time = time.time()

In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import csv
import pandas as pd
from bs4 import BeautifulSoup as bs4
import re

In [16]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.headless = True
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - There is no [win32] chromedriver for browser 102.0.5005 in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\saidu\.wdm\drivers\chromedriver\win32\102.0.5005.61]


In [17]:
# Saving Query Data from csv to list
def savingQueryData(csvFile):
    queryDataCsv= pd.read_csv(csvFile)
    queryList = [i for i in queryDataCsv["query"]]
    locationList = [i for i in queryDataCsv["location"]]
    return queryList, locationList
    
def savingLoginData(csvFile):
    loginDataCsv = pd.read_csv(csvFile)
    emailList = [i for i in loginDataCsv["email"]]
    passwordList = [i for i in loginDataCsv["password"]]
    return emailList, passwordList

# Return searchable facebook url
def facebook_search_url(queryList, locationList):
    search_url_list = []
    for j in range(len(queryList)):
        splitQuery = queryList[j].split(" ")
        splitLocation = locationList[j].split(" ")

        searchUrl = "https://www.facebook.com/search/pages/?q="
        for i in splitQuery:
            searchUrl = searchUrl + i + "%20"
        for i in splitLocation:
            searchUrl = searchUrl + i + "%20"
        search_url_list.append(searchUrl)
    return search_url_list

queryList, locationList = savingQueryData("queries.csv")
search_url_list = facebook_search_url(queryList, locationList)

search_url_list

['https://www.facebook.com/search/pages/?q=car%20repair%20california%20',
 'https://www.facebook.com/search/pages/?q=car%20repair%20new%20york%20',
 'https://www.facebook.com/search/pages/?q=car%20repair%20canada%20',
 'https://www.facebook.com/search/pages/?q=house%20canada%20',
 'https://www.facebook.com/search/pages/?q=house%20new%20york%20',
 'https://www.facebook.com/search/pages/?q=hotel%20%20boston%20']

In [18]:
def generalInfo_aboutPage():
    soup = bs4(driver.page_source, "html.parser")
    list_info = soup.find_all("div", class_="je60u5p8")

    info = ""
    for idx, j in enumerate(list_info):
        for i in list_info[idx].findAll("span"):
            if not i.text in info:
                info = info + i.text + " "
        info = info.replace("General", "")
    return info

def collectURL(string):
    # findall() has been used 
    # with valid conditions for urls in string
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)      
    return [x[0] for x in url]

def collectEMAIL(string):
    match = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', string)
    return match
def collectPHONE(string):
    match = re.findall(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})', string)
    return match

def mainScraper():
    #Html of about page
    html = driver.page_source
    soup = bs4(html, "html.parser")
    target_listing = soup.find_all("div", class_="sjgh65i0")

    master_list = []
    for idx,i in enumerate(target_listing):
    # Single Listing
        data_dict={}
        try:
            page_name = i.a["aria-label"]
            facebook_link = i.a["href"]
            data_dict["Page Name"] = page_name
            data_dict["Facebook Link"] = facebook_link
            facebook_link_aboutPage = facebook_link.replace("?__tn__=%3C", "about")
            if not "/about" in facebook_link_aboutPage:
                facebook_link_aboutPage = facebook_link_aboutPage.replace("about", "/about")
            print(facebook_link_aboutPage, "\n")
            if "facebook.com" in facebook_link_aboutPage:
                driver.get(facebook_link_aboutPage)
            time.sleep(5)
            #Accessing each lead's about page
            data_dict["Url"] = "".join(collectURL(generalInfo_aboutPage()))
            data_dict["Email"] = "".join(collectEMAIL(generalInfo_aboutPage()))
            data_dict["Phone Numer"] = "".join(collectPHONE(generalInfo_aboutPage()))
            data_dict["General Information"] = generalInfo_aboutPage()
        except:
            print("Error")
        master_list.append(data_dict)
        time.sleep(5)
        print(f"Percentage done: {idx/len(target_listing)*100}%")
    df = pd.DataFrame(master_list)
    return df
    

In [19]:
emailList, passwordList = savingLoginData("login.csv")

driver.get("https://www.facebook.com")
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

username.clear()
username.send_keys(emailList[0])
password.clear()
password.send_keys(passwordList[0])

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [22]:
time.sleep(5)
for idx,url in enumerate(search_url_list):
    driver.get(url)
    time.sleep(2)
    for i in range(20):
        time.sleep(2)
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") 
#     TODO: Solution for House Canda Sraping - Posts/Pages/Group ?
    time.sleep(2)
    df = mainScraper()
    queryList, locationList = savingQueryData("queries.csv")
    df.to_csv(f"{queryList[idx]} {locationList[idx]}.csv")
    break

KeyboardInterrupt: 

In [140]:
driver.quit()

In [141]:
print("Time Taken",  (time.time() - start_time)/60, " min")

Time Taken 28.87259211540222  min


In [21]:
df

,Page Name,Facebook Link,Url,Email,Phone Numer,General Information
0,Car Repair + Rental Dhaka,https://web.facebook.com/recarbd/?__tn__=%3C,http://recardbd.com/,,621-9966,"3,607 people like this 3,749 people follow th..."
1,HASAN AUTO REPAiR,https://web.facebook.com/hasanautorepair/?__tn...,,,788-7887,236 people like this 243 people follow this A...
2,Car Heaven Workshop & Servicing Centre,https://web.facebook.com/CarHeavenWorkshopServ...,http://www.carheaven.com/,bdcarheaven@gmail.com,710-4160,"8,696 people like this 8,739 people follow th..."
3,Car Paint BD,https://web.facebook.com/carpaintbd/?__tn__=%3C,https://sites.google.com/view/carpaintbd,carpaintbd2@gmail.com,848-0891880 1848,285 people like this 313 people follow this A...
4,Car Karigor,https://web.facebook.com/CarKarigor/?__tn__=%3C,https://www.carkarigor.com/,info@carkarigor.com,712-0499,"22,074 people like this including 1 of your f..."
...,...,...,...,...,...,...
98,Car Mods BD,https://web.facebook.com/CarModsBangladesh/?__...,https://carmodsbd.com/,info@carmodsbd.com,517-0358880 1517,"55,640 people like this 62,343 people follow ..."
99,Hybrid auto care bd,https://web.facebook.com/Hybrid-auto-care-bd-1...,,,617-6848,"10,306 people like this 10,366 people follow ..."
100,Top Gear Auto Service BD Limited,https://web.facebook.com/Top-Gear-Auto-Service...,http://www.TopGear-BD.com/,contact@topgear-bd.com,733-1100,"1,715 people like this 1,766 people follow th..."
101,Pacific Car Center,https://web.facebook.com/pacificcarcenter/?__t...,http://www.pacificcarcenter.com/,,730-3021,"36,307 people like this 36,304 people follow ..."
